## Step 1: Acquire Data (Database Version)

For this experiment, we selected the Breast Cancer Wisconsin dataset from the UCI Machine Learning Repository.

The dataset was first downloaded as a CSV file and uploaded to a Neon PostgreSQL database.

If a Neon connection string is not available, the dataset will automatically load from the local CSV file to ensure reproducibility.
## Dataset Source

UCI Machine Learning Repository  
Breast Cancer Wisconsin (Diagnostic) Dataset  

Original data file:  
https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.data  

The dataset was downloaded directly from the UCI repository and saved locally as a CSV file for analysis.



In [1]:
# Import required libraries
import os
import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import create_engine

# Load environment variables from .env file
load_dotenv()

# Get Neon database URL
DATABASE_URL = os.getenv("DATABASE_URL")

# Check if environment variable is loaded
print("DATABASE_URL loaded successfully:", DATABASE_URL is not None)


DATABASE_URL loaded successfully: True


In [2]:
# Load raw dataset without column names

df_raw = pd.read_csv("data/breast_cancer_wisconsin.csv", header=None)

print("Raw dataset loaded.")
print("Shape:", df_raw.shape)

df_raw.head()


Raw dataset loaded.
Shape: (569, 32)


,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [3]:
# Assign column names based on UCI documentation

column_names = [
    "id",
    "diagnosis",
    "radius_mean", "texture_mean", "perimeter_mean", "area_mean", "smoothness_mean",
    "compactness_mean", "concavity_mean", "concave_points_mean", "symmetry_mean", "fractal_dimension_mean",
    "radius_se", "texture_se", "perimeter_se", "area_se", "smoothness_se",
    "compactness_se", "concavity_se", "concave_points_se", "symmetry_se", "fractal_dimension_se",
    "radius_worst", "texture_worst", "perimeter_worst", "area_worst", "smoothness_worst",
    "compactness_worst", "concavity_worst", "concave_points_worst", "symmetry_worst", "fractal_dimension_worst"
]

df_raw.columns = column_names

print("Column names assigned successfully.")
df_raw.head()


Column names assigned successfully.


,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave_points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave_points_worst,symmetry_worst,fractal_dimension_worst
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


The original UCI dataset file does not contain column headers.
Therefore, we manually assigned feature names based on the official
UCI Breast Cancer Wisconsin (Diagnostic) dataset documentation.

This step ensures:
- Proper feature identification
- Improved readability
- Compatibility with machine learning workflows


In [4]:
# Upload dataset to Neon PostgreSQL database

# This step stores the dataset in a cloud database (Neon)
# so that future analysis can be performed directly from the cloud.

if DATABASE_URL:
    
    # Create database engine connection
    engine = create_engine(DATABASE_URL)
    
    # Upload dataframe to Neon
    # if_exists="replace" overwrites table if it already exists
    df_raw.to_sql(
        "breast_cancer_knn",
        engine,
        if_exists="replace",
        index=False
    )
    
    print("Dataset successfully uploaded to Neon database.")

else:
    print("DATABASE_URL not found. Cannot upload to Neon.")


Dataset successfully uploaded to Neon database.


In [5]:
# Load dataset from Neon (if available)
# Otherwise fallback to local CSV

if DATABASE_URL:
    try:
        engine = create_engine(DATABASE_URL)
        
        df = pd.read_sql(
            "SELECT * FROM breast_cancer_knn;",
            engine
        )
        
        print("Dataset successfully loaded from Neon.")
        
    except Exception as e:
        print("Neon connection failed. Loading local CSV instead.")
        df = df_raw.copy()
        
else:
    print("DATABASE_URL not found. Loading local CSV.")
    df = pd.read_csv("data/breast_cancer_wisconsin.csv", header=None)

print("Dataset shape:", df.shape)
df.head()


Dataset successfully loaded from Neon.
Dataset shape: (569, 32)


,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave_points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave_points_worst,symmetry_worst,fractal_dimension_worst
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In this step, we successfully verified that the dataset was uploaded to the Neon PostgreSQL cloud database.

The notebook connects to Neon using the `DATABASE_URL` environment variable and retrieves the table `breast_cancer_knn`.

The output confirms:

- ✅ Successful database connection  
- ✅ Table exists in Neon  
- ✅ Dataset shape matches the original CSV (569 rows × 32 columns)

This ensures that all subsequent modeling steps will use the dataset directly from the cloud database, fulfilling the workshop requirement of acquiring data from a database.

If the database connection is not available, the notebook is designed to optionally fall back to the local CSV file to maintain reproducibility.


## Step 2 — Preprocess Data

In [6]:
#  Check Missing Values

# Count missing values per column
missing_per_column = df.isnull().sum()

# Total missing values in dataset
total_missing = missing_per_column.sum()

print("Missing values per column:")
print(missing_per_column)

print("\nTotal missing values:", total_missing)


Missing values per column:
id                         0
diagnosis                  0
radius_mean                0
texture_mean               0
perimeter_mean             0
area_mean                  0
smoothness_mean            0
compactness_mean           0
concavity_mean             0
concave_points_mean        0
symmetry_mean              0
fractal_dimension_mean     0
radius_se                  0
texture_se                 0
perimeter_se               0
area_se                    0
smoothness_se              0
compactness_se             0
concavity_se               0
concave_points_se          0
symmetry_se                0
fractal_dimension_se       0
radius_worst               0
texture_worst              0
perimeter_worst            0
area_worst                 0
smoothness_worst           0
compactness_worst          0
concavity_worst            0
concave_points_worst       0
symmetry_worst             0
fractal_dimension_worst    0
dtype: int64

Total missing values: 0


## Missing Value Inspection

We checked the dataset for missing values.

The results show that all features contain zero missing values.

Therefore, no imputation is required, and the dataset is clean for modeling.


In [7]:
# Remove ID Column

df_model = df.drop(columns=["id"])

print("ID column removed.")
df_model.head()


ID column removed.


,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave_points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave_points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


## Remove ID Column

The `id` column is removed because it does not contain predictive information.
It is only an identifier and should not be used in model training.


In [8]:
# Encode Target Variable


df_model["diagnosis"] = df_model["diagnosis"].map({"M": 1, "B": 0})

print("Diagnosis encoded successfully.")
print(df_model["diagnosis"].value_counts())

df_model.head()


Diagnosis encoded successfully.
diagnosis
0    357
1    212
Name: count, dtype: int64


,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave_points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave_points_worst,symmetry_worst,fractal_dimension_worst
0,1,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,1,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,1,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,1,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


## Target Encoding

The `diagnosis` variable is converted to numeric format:

- M (Malignant) → 1  
- B (Benign) → 0  

This transformation is required because machine learning models
operate on numerical data.


In [9]:
# Separate Features and Target

X = df_model.drop(columns=["diagnosis"])
y = df_model["diagnosis"]

print("Feature matrix shape:", X.shape)
print("Target shape:", y.shape)


Feature matrix shape: (569, 30)
Target shape: (569,)


In [10]:
from sklearn.model_selection import train_test_split

# Split dataset into training and test sets

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42
)

print("Training set:", X_train.shape)
print("Test set:", X_test.shape)


Training set: (455, 30)
Test set: (114, 30)


In [11]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

# Fit scaler on training data only
X_train_scaled = scaler.fit_transform(X_train)

# Transform test data
X_test_scaled = scaler.transform(X_test)

print("Feature scaling completed.")


Feature scaling completed.


## Step 2 — Data Preprocessing Completed

In this step, we prepared the dataset for machine learning:

- Verified that there are no missing values
- Encoded the categorical target variable (diagnosis)
- Removed the unnecessary ID column
- Split the dataset into training and test sets
- Applied feature scaling using StandardScaler

The data is now ready for model training.


## Step 3 — Implement KNN using Pipeline Pattern

In [12]:
# KNN Model with Pipeline

from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Create a pipeline:
#  1 → StandardScaler
#  2 → KNN classifier

knn_pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("knn", KNeighborsClassifier(n_neighbors=5))
])

# Train model
knn_pipeline.fit(X_train, y_train)

# Predict on test set
y_pred = knn_pipeline.predict(X_test)

# Evaluate performance
accuracy = accuracy_score(y_test, y_pred)

print("Model training completed.")
print("Accuracy:", accuracy)


Model training completed.
Accuracy: 0.9473684210526315


In [13]:
# Model Evaluation

# Print classification report
# This shows precision, recall, f1-score, and support for each class
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Print confusion matrix
# This shows how many samples were correctly and incorrectly classified
# Format:
# [[True Negatives, False Positives]
#  [False Negatives, True Positives]]
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))



Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.96      0.96        71
           1       0.93      0.93      0.93        43

    accuracy                           0.95       114
   macro avg       0.94      0.94      0.94       114
weighted avg       0.95      0.95      0.95       114

Confusion Matrix:
[[68  3]
 [ 3 40]]


## Model Evaluation

The KNN model achieved an accuracy of approximately 94.7%.

The classification report shows strong precision and recall for both classes.

The confusion matrix indicates that only a small number of samples were misclassified.

Overall, KNN performs well on this dataset.


# KNN Engineering Version — Pipeline Pattern Implementation

This section implements KNN using the Pipeline Pattern, following the workshop requirement:

> "Chain them together in a clear pipeline."



In [14]:
# Step 1 — load_data()

def load_data():
    """
    Load dataset from Neon PostgreSQL.
    If database connection fails, fallback to local CSV.
    """

    try:
        engine = create_engine(DATABASE_URL)

        df = pd.read_sql(
            "SELECT * FROM breast_cancer_knn;",
            engine
        )

        print("Data loaded from Neon database.")
        return df

    except Exception:
        print("Neon connection failed. Loading local CSV instead.")

        df = pd.read_csv(
            "data/breast_cancer_wisconsin.csv",
            header=None
        )

        return df


In [15]:
# Step 2 — preprocess_data()

def preprocess_data(df):
    """
    Preprocess dataset:
    - Encode target variable
    - Split into train/test
    - Scale numerical features
    """

    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import StandardScaler

    # Encode diagnosis column
    # M → 1, B → 0

    df["diagnosis"] = df["diagnosis"].map({"M": 1, "B": 0})

    # Drop ID column
    
    X = df.drop(["id", "diagnosis"], axis=1)
    y = df["diagnosis"]

    # Train/test split
    X_train, X_test, y_train, y_test = train_test_split(
        X,
        y,
        test_size=0.2,
        random_state=42
    )

    # Feature scaling
    
    scaler = StandardScaler()

    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    print("Preprocessing completed.")
    print("Training shape:", X_train_scaled.shape)
    print("Test shape:", X_test_scaled.shape)

    return X_train_scaled, X_test_scaled, y_train, y_test


This function performs all preprocessing steps:
- Separate features and target variable
- Encode categorical labels
- Split dataset into training and testing sets
- Scale numerical features

This keeps preprocessing modular and reusable.


In [16]:
# Step 3 — train_model()

def train_model(X_train, y_train, n_neighbors=5):
    """
    Train KNN classifier.
    
    Parameters:
    - X_train: Scaled training features
    - y_train: Training labels
    - n_neighbors: Number of neighbors (default=5)
    
    Returns:
    - Trained KNN model
    """

    from sklearn.neighbors import KNeighborsClassifier

    # Initialize model
    model = KNeighborsClassifier(n_neighbors=n_neighbors)

    # Fit model
    model.fit(X_train, y_train)

    print("Model training completed.")

    return model


This function trains the K-Nearest Neighbors (KNN) model 
using the preprocessed training data.

The function only handles model training 
to maintain modular design.


In [17]:
# Step 4 — evaluate_model()

def evaluate_model(model, X_test, y_test):
    """
    Evaluate trained KNN model.
    
    Parameters:
    - model: Trained KNN model
    - X_test: Scaled test features
    - y_test: Test labels
    
    Returns:
    - None (prints evaluation results)
    """

    from sklearn.metrics import accuracy_score
    from sklearn.metrics import classification_report
    from sklearn.metrics import confusion_matrix

    # Make predictions
    y_pred = model.predict(X_test)

    # Accuracy
    accuracy = accuracy_score(y_test, y_pred)

    print("Evaluation Results")
    print("------------------")
    print("Accuracy:", accuracy)

    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))

    print("Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred))


This function evaluates the trained KNN model 
using the test dataset.

It calculates:
- Accuracy
- Classification Report
- Confusion Matrix

This step is separated from training 
to maintain modular and clean pipeline structure.


## Step 4 — Train and Evaluate KNN

In this step, we:
- Split data into training and test sets
- Fit the KNN model
- Evaluate performance using accuracy and classification metrics


In [18]:
# Main Pipeline Execution

# Step 1 — Load data
df = load_data()

# Step 2 — Preprocess
X_train, X_test, y_train, y_test = preprocess_data(df)

# Step 3 — Train
model = train_model(X_train, y_train)

# Step 4 — Evaluate
evaluate_model(model, X_test, y_test)


Data loaded from Neon database.
Preprocessing completed.
Training shape: (455, 30)
Test shape: (114, 30)
Model training completed.
Evaluation Results
------------------
Accuracy: 0.9473684210526315

Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.96      0.96        71
           1       0.93      0.93      0.93        43

    accuracy                           0.95       114
   macro avg       0.94      0.94      0.94       114
weighted avg       0.95      0.95      0.95       114

Confusion Matrix:
[[68  3]
 [ 3 40]]


The evaluation confirms that the KNN model achieves approximately 94.7% accuracy on the test dataset, indicating strong predictive performance.


## Final Summary

In this exercise, we implemented KNN using the Pipeline Pattern.

The workflow includes:
- Data acquisition from Neon PostgreSQL
- Fallback mechanism to local CSV
- Data preprocessing (splitting & scaling)
- KNN model training
- Model evaluation

All components are structured into modular functions
and chained together in a clear execution pipeline.

This implementation follows clean engineering practices
and satisfies the workshop requirement.


## Step 5 — Compare Results with Peers (Talking Points)

### Dataset Choice
Our team selected the Breast Cancer Wisconsin (Diagnostic) dataset from the UCI Machine Learning Repository and stored it in a Neon PostgreSQL cloud database.

The dataset contains:
- 569 samples
- 30 numerical features
- 1 categorical target variable (diagnosis: malignant or benign)

This makes it suitable for a KNN classification task.

---

### Preprocessing Decisions

We performed the following preprocessing steps:

- Checked for missing values (none found)
- Split data into training (80%) and test (20%) sets
- Applied StandardScaler to normalize numerical features

Scaling was especially important because KNN is distance-based and sensitive to feature magnitude differences.

Different teams may choose:
- Different test sizes
- Different scaling methods (MinMaxScaler vs StandardScaler)
- Additional feature selection

These decisions can affect model performance.

---

### Model Results

Our KNN model achieved:

- Accuracy: ~94.7%
- Balanced precision and recall for both classes
- A low number of misclassifications (6 total errors)

This indicates strong classification performance on the dataset.

---

### Why Results May Differ Across Teams

Results may vary due to:

- Different random_state values
- Different K values
- Different preprocessing strategies
- Different scaling approaches
- Feature selection or dimensionality reduction

Even small preprocessing differences can influence KNN results.

---

### Was KNN the Best Choice?

KNN performs well on this dataset because:

- The dataset is relatively small
- Features are numerical and well-structured
- Classes are fairly separable

However, KNN can be sensitive to:
- Feature scaling
- Choice of K
- High-dimensional data

Other models such as Logistic Regression or SVM might also perform well and could be compared in future experiments.
